# Beginning

Fow now it is almost just a copy/paste from https://github.com/bradleyemi/coupled-ldpc/blob/master/ldpc.py

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pickle
import scipy.io

Let's load two specific generator and parity check matrices ${\cal H}$, ${\cal G}$, respectively.

In [15]:
def loadLDPC(name):
    """
    :param - name: the name of the file containing LDPC matrices
  
    return values:
    G: generator matrix
    H: parity check matrix
    """
    A = scipy.io.loadmat(name)
    G = A['G']
    H = A['H']
    return G, H

In [16]:
G, H = loadLDPC('codes/ldpc36-128.mat')

In [20]:
print(G)
print(G.shape)

[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [1 0 1 ... 1 0 0]
 [1 1 0 ... 1 1 1]]
(256, 128)


In [24]:
print(H)
print(H.shape)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]]
(128, 256)


Defining a factor.

In [26]:
class Factor(object):
    def __init__(self, node_type, id_, neighbors):
        self.node_type = node_type
        self.id = id_
        self.neighbors = neighbors

A class `Graph` to hold the factor graph structure of H and initialize messages.

In [32]:
class Graph(object):
    def __init__(self, H):
        self.H = H

        self.n_vars = H.shape[1]
        self.n_checks = H.shape[0]

        self.vars = []
        self.checks = []

        for i in range(self.n_vars):
            print('Constructing variable factor', i)
            
            # Given the i-th variable, collect all its variables 'neighbours'.
            neighbors = np.nonzero(H[:,i])
            
            # But the graph is iteratively constructed using only the n.n.
            self.vars.append(Factor('var', i, neighbors[0]))

        for i in range(self.n_checks):
            print('Constructing check factor', i)
            
            # Given the i-th factor, we collect all its factor 'neighbours'.
            neighbors = np.nonzero(H[i,:])
            # But the graph is iteratively constructed using only the n.n.
            self.checks.append(Factor('check', i, neighbors[0]))

A class `ClusterGraph` to hold the factor graph structure of H and initialize messages.

In [37]:
class ClusterGraph(object):
    def __init__(self, graph, codeword):
        
        self.graph = graph
        self.orig_codeword = codeword
        self.decoded_codeword = copy(codeword)

        self.var_to_check_msgs = {}
        self.check_to_var_msgs = {}

        for var in self.graph.vars:
            for neighbor in var.neighbors:
                self.var_to_check_msgs[(var.id_, neighbor)] = -1 #Message("var_to_check", -1)

        for check in self.graph.checks:
            for neighbor in check.neighbors:
                self.check_to_var_msgs[(check.id_, neighbor)] = -1 #Message("check_to_var", -1)

    def decode(self, iterations, yield_errors=False):
        
        errors = len([c for c in self.decoded_codeword if c == -1])
        
        print("decoding for error probability", float(errors) / self.graph.n_vars()
              
        for iteration in xrange(iterations):
              
            # First iterate over var -> check messages
            for source_dest, msg in self.var_to_check_msgs.iteritems():
              
                source = source_dest[0]
                dest = source_dest[1]
                source_neighbors = self.graph.vars[source].neighbors
              
                in_msgs = [self.check_to_var_msgs[(neigh, source)] for neigh in source_neighbors if neigh != dest]
              
                # If any incoming message from a check is not -1, we know the variable must be equal to the message value
                # so we can pass it forward and decode the variable bit
                for in_msg in in_msgs:
                    if in_msg != -1:
                        self.var_to_check_msgs[source_dest] = in_msg
                        self.decoded_codeword[source] = msg
                        break
              
                if self.decoded_codeword[source] != -1:
                    self.var_to_check_msgs[source_dest] = self.decoded_codeword[source]
            
            # Now iterate over check -> var messages
            for source_dest, msg in self.check_to_var_msgs.iteritems():
              
                source = source_dest[0]
                dest = source_dest[1]
                source_neighbors = self.graph.checks[source].neighbors
              
                in_msgs = [self.var_to_check_msgs[(neigh, source)] for neigh in source_neighbors if neigh != dest]
              
                # A check must have all incoming messages non-erased to be certain of the value of the destination
                # so -1 cannot be in the incoming messages. 
                # If all incoming messages are known, pass value that makes parity check = 0 (mod 2) and decode bit
                
              if -1 not in in_msgs:
                    self.check_to_var_msgs[source_dest] = sum(in_msgs) % 2
                    self.decoded_codeword[dest] = sum(in_msgs) % 2
            new_errors = len([c for c in self.decoded_codeword if c == -1])
              
            if yield_errors:
                yield self.decoded_codeword
            if (new_errors == errors) or (new_errors == 0):
                break
            else:
                errors = new_errors
        #return float(errors) / self.graph.H.shape[1]
